In [ ]:
import requests
from urllib.parse import urlparse, urljoin
from bs4 import BeautifulSoup
import colorama
from requests import get
from requests.exceptions import RequestException  #pip3 install requests
from contextlib import closing
from bs4 import BeautifulSoup

In [ ]:
colorama.init()
GREEN = colorama.Fore.GREEN
GRAY = colorama.Fore.LIGHTBLACK_EX
RESET = colorama.Fore.RESET

In [ ]:
# initialize the set of links (unique links)
internal_urls = set()
external_urls = set()

In [ ]:
def is_valid(url):
    """
    Checks whether `url` is a valid URL.
    """
    parsed = urlparse(url)
    return bool(parsed.netloc) and bool(parsed.scheme)

In [ ]:
def get_all_website_links(url):
    """
    Returns all URLs that is found on `url` in which it belongs to the same website
    """
    # all URLs of `url`
    urls = set()
    # domain name of the URL without the protocol
    domain_name = urlparse(url).netloc
    soup = BeautifulSoup(requests.get(url).content, "html.parser")
    
    for a_tag in soup.findAll(lambda tag: tag.name == 'a'):
    
        href = a_tag.attrs.get("href")
        
        if href == "" or href is None:
            # href empty tag
            continue
        # join the URL if it's relative (not absolute link)
        
        href = urljoin(url, href)
        parsed_href = urlparse(href)
        # remove URL GET parameters, URL fragments, etc.
        href = parsed_href.scheme + "://" + parsed_href.netloc + parsed_href.path
        
        if not is_valid(href):
            # not a valid URL
            continue
            
        if href in internal_urls:
            # already in the set
            continue
            
        if domain_name not in href:
            # external link
            if href not in external_urls:
                print(f"{GRAY}[!] External link: {href}{RESET}")
                external_urls.add(href)
            continue
            
        print(f"{GREEN}[*] Internal link: {href}{RESET}")
        urls.add(href)
        internal_urls.add(href)
        
    return urls

In [ ]:
# number of urls visited so far will be stored here
total_urls_visited = 0

def crawl(url, max_urls = 2000):
    """
    Crawls a web page and extracts all links.
    You'll find all links in `external_urls` and `internal_urls` global set variables.
    params:
        max_urls (int): number of max urls to crawl, default is 30.
    """
    global total_urls_visited
    total_urls_visited += 1
    links = get_all_website_links(url)
    for link in links:
        if total_urls_visited > max_urls:
            break
        crawl(link, max_urls=max_urls)

In [ ]:
import time

t0 = time.time()

In [ ]:
crawl('https://www.forever21.ca/collections/women/')

In [ ]:
t1 = time.time()
total_time = t1-t0
print(total_time)

In [ ]:
import re

extracted_url = list(internal_urls)
r = re.compile("https://www.forever21.ca/collections/women/")
url_list = list(filter(r.match, extracted_url))

In [ ]:
print(url_list)

In [ ]:
print(len(url_list))

In [ ]:
url_list_2 = url_list[1:5]

In [ ]:
url_list_2

In [ ]:
def is_good_response(resp):

    #Returns true if the response seems to be HTML, false otherwise

    content_type = resp.headers['Content-Type'].lower()
    return (resp.status_code == 200 
            and content_type is not None 
            and content_type.find('html') > -1)

In [ ]:
def simple_get(url):
    """
    Attempts to get the content at `url` by making an HTTP GET request.
    If the content-type of response is some kind of HTML/XML, return the
    text content, otherwise return None.
    """
    try:
        with closing(get(url, stream=True)) as resp:
            if is_good_response(resp):
                return resp.content
            else:
                return None

    except RequestException as e:
        log_error('Error during requests to {0} : {1}'.format(url, str(e)))
        return None

In [ ]:
t0 = time.time()

In [ ]:
descr = []

for url in url_list:
    response = simple_get(url)
    html = BeautifulSoup(response, 'html.parser')
    descr.append(html.findAll("div", {"class": "product__description-toggle-item resp-tab-content-active"}))

In [ ]:
t1 = time.time()
total_time = t1-t0
print(total_time)

In [ ]:
descr